In [2]:
import os
import json
import torch
import faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

d:\anconda\envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Step 1: Load or prepare HumanEval dataset
# ------------------------------------------------
def load_humaneval_dataset(path="data/humaneval.jsonl"):
    if not os.path.exists(path):
        print("Dataset not found locally. Downloading from HuggingFace...")
        dataset = load_dataset("openai/openai_humaneval", split="test")
        os.makedirs("data", exist_ok=True)
        with open(path, "w", encoding="utf-8") as f:
            for item in dataset:
                record = {
                    "task_id": item["task_id"],
                    "prompt": item["prompt"],
                    "solution": item["canonical_solution"],
                }
                f.write(json.dumps(record) + "\n")
        print(f"✅ Saved {len(dataset)} tasks to {path}")

    with open(path, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f]
    print(f"✅ Loaded {len(data)} HumanEval tasks.")
    return data


In [4]:
# Step 2: Build embeddings + FAISS index
# ------------------------------------------------
def build_embeddings_and_faiss(
    data,
    embed_model="sentence-transformers/all-MiniLM-L6-v2",
    index_path="data/faiss.index"
):
    os.makedirs("data", exist_ok=True)
    print("\n🔹 Loading embedding model...")
    model = SentenceTransformer(embed_model)

    prompts = [d["prompt"] for d in data]
    print(f"🔹 Creating embeddings for {len(prompts)} tasks...")
    embeddings = model.encode(prompts, show_progress_bar=True, convert_to_numpy=True)

    print("🔹 Building FAISS index...")
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    faiss.write_index(index, index_path)
    print(f"✅ FAISS index saved to {index_path}")

    return model, index

In [5]:
# Step 3: Retrieve similar examples
# ------------------------------------------------
def retrieve_examples(query, embedder, index, data, top_k=3):
    print("\n🔹 Retrieving similar code examples...")
    query_vector = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_vector, top_k)
    results = []
    for idx in indices[0]:
        # Handles both key names safely
        results.append(data[idx].get("solution", data[idx].get("canonical_solution", "")))
    return results



In [6]:
# Step 4: Generate code with open model
# ------------------------------------------------
def generate_code(query, retrieved_examples,
                  model_name="Salesforce/codegen-350M-multi", #"HuggingFaceH4/starchat-alpha"
                  max_new_tokens=256):
    print(f"\n🔹 Loading generation model: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float32)

    if torch.cuda.is_available():
        model = model.to("cuda")

    context = "\n\n".join(retrieved_examples)
    prompt = f"# Task:\n{query}\n\n# Related Examples:\n{context}\n\n# Solution:\n"

    inputs = tokenizer(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, temperature=0.3)

    code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\n✅ Generated Code:\n")
    print(code)
    return code


In [7]:
# Step 5: Full RAG pipeline
# ------------------------------------------------
def rag_pipeline(task_description):
    data = load_humaneval_dataset("data/humaneval.jsonl")

    index_path = "data/faiss.index"
    if os.path.exists(index_path):
        print("🔹 Loading existing FAISS index...")
        embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
        index = faiss.read_index(index_path)
    else:
        embedder, index = build_embeddings_and_faiss(data)

    retrieved = retrieve_examples(task_description, embedder, index, data)
    generated_code = generate_code(task_description, retrieved)
    return generated_code


In [7]:
# Run Example
# ------------------------------------------------
if __name__ == "__main__":
    task_query = "Write a Python function that checks if a given string is a palindrome."
    rag_pipeline(task_query)


✅ Loaded 164 HumanEval tasks.
🔹 Loading existing FAISS index...

🔹 Retrieving similar code examples...

🔹 Loading generation model: Salesforce/codegen-350M-multi


d:\anconda\envs\rag\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Alieldin\.cache\huggingface\hub\models--Salesforce--codegen-350M-multi. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
`torch_dtype` is deprecated! Use `dtype` instead!
Xet Storage is enabled for this repo, but the 'hf_xet' package is 


✅ Generated Code:

# Task:
Write a Python function that checks if a given string is a palindrome.

# Related Examples:
for i in range(len(text)):
        if text[i] != text[len(text) - 1 - i]:
            return False
    return True

if not string:
        return ''

    beginning_of_suffix = 0

    while not is_palindrome(string[beginning_of_suffix:]):
        beginning_of_suffix += 1

    return string + string[:beginning_of_suffix][::-1]

s = ''.join([char for char in s if char not in c])
    return (s,s[::-1] == s)

# Solution:
#
#     1. Find the longest palindrome in the string
#     2. Find the longest palindrome in the string
#     3. Find the longest palindrome in the string
#     4. Find the longest palindrome in the string
#     5. Find the longest palindrome in the string
#     6. Find the longest palindrome in the string
#     7. Find the longest palindrome in the string
#     8. Find the longest palindrome in the string
#     9. Find the longest palindrome in the string

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Salesforce/codegen-350M-multi"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.save_pretrained("models/codegen350")
tokenizer.save_pretrained("models/codegen350")

'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 15f33314-8afe-40cb-a39f-c2fa591f2b60)')' thrown while requesting HEAD https://huggingface.co/Salesforce/codegen-350M-multi/resolve/main/config.json
Retrying in 1s [Retry 1/5].
Some weights of the model checkpoint at Salesforce/codegen-350M-multi were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal

('models/codegen350\\tokenizer_config.json',
 'models/codegen350\\special_tokens_map.json',
 'models/codegen350\\vocab.json',
 'models/codegen350\\merges.txt',
 'models/codegen350\\added_tokens.json',
 'models/codegen350\\tokenizer.json')

In [8]:
# --- Test RAG System ---
from pprint import pprint

def test_rag_queries():
    queries = [
        "Write a Python function that returns the Fibonacci sequence up to n.",
        "Write a function to check if a number is prime.",
        "Implement a function that sorts a list of integers using bubble sort.",
        "Write a function that counts the number of vowels in a string.",
        "Create a Python function that reverses a linked list."
    ]

    for q in queries:
        print("\n" + "="*80)
        print(f"🧩 Task: {q}")
        print("="*80)

        # Run the RAG pipeline
        generated = rag_pipeline(q)

        # Show final output (trimmed)
        print("\n✅ Generated Code (preview):\n")
        print(generated[:800])  # show first 800 chars only
        print("\n--- END OF OUTPUT ---\n")


if __name__ == "__main__":
    test_rag_queries()



🧩 Task: Write a Python function that returns the Fibonacci sequence up to n.
✅ Loaded 164 HumanEval tasks.
🔹 Loading existing FAISS index...

🔹 Retrieving similar code examples...

🔹 Loading generation model: Salesforce/codegen-350M-multi


`torch_dtype` is deprecated! Use `dtype` instead!
Some weights of the model checkpoint at Salesforce/codegen-350M-multi were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trai


✅ Generated Code:

# Task:
Write a Python function that returns the Fibonacci sequence up to n.

# Related Examples:
if n == 0:
        return 0
    if n == 1:
        return 1
    return fib(n - 1) + fib(n - 2)

results = [0, 0, 2, 0]
    if n < 4:
        return results[n]

    for _ in range(4, n + 1):
        results.append(results[-1] + results[-2] + results[-3] + results[-4])
        results.pop(0)

    return results[-1]

if n == 0:
        return 0
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fibfib(n - 1) + fibfib(n - 2) + fibfib(n - 3)

# Solution:
#
#     fib(n - 1) + fib(n - 2) + fib(n - 3)
#     fibfib(n - 1) + fibfib(n - 2) + fibfib(n - 3)
#     fibfibf(n - 1) + fibfibf(n - 2) + fibfibf(n - 3)
#     fibfibf(n - 1) + fibfibf(n - 2) + fibfibf(n - 3)
#     fibfibf(n - 1) + fibfibf(n - 2) + fibfibf(n - 3)
#     fibfibf(n - 1) + fibfibf(n - 2) + fibfibf(n - 3)
#     fibfibf(n - 1) + fibfibf(n - 2) + fibfibf(n - 3)
#     fibfibf(n - 1) + fibfibf(n

Some weights of the model checkpoint at Salesforce/codegen-350M-multi were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (


✅ Generated Code:

# Task:
Write a function to check if a number is prime.

# Related Examples:
if n < 2:
        return False
    for k in range(2, n - 1):
        if n % k == 0:
            return False
    return True

def is_prime(n):
        for j in range(2,n):
            if n%j == 0:
                return False
        return True

    for i in range(2,101):
        if not is_prime(i): continue
        for j in range(2,101):
            if not is_prime(j): continue
            for k in range(2,101):
                if not is_prime(k): continue
                if i*j*k == a: return True
    return False

l = len(string)
    if l == 0 or l == 1:
        return False
    for i in range(2, l):
        if l % i == 0:
            return False
    return True

# Solution:
#
#     1. Find the number of prime factors of n.
#     2. Find the number of prime factors of n.
#     3. Find the number of prime factors of n.
#     4. Find the number of prime factors of n.
#     5. Find the nu

...


✅ Generated Code:

# Task:
Implement a function that sorts a list of integers using bubble sort.

# Related Examples:
return [] if len(array) == 0 else sorted(array, reverse= (array[0]+array[-1]) % 2 == 0)

def digits_sum(n):
        neg = 1
        if n < 0: n, neg = -1 * n, -1 
        n = [int(i) for i in str(n)]
        n[0] = n[0] * neg
        return sum(n)
    return sorted(nums, key=digits_sum)

res, switch = [], True
    while lst:
        res.append(min(lst) if switch else max(lst))
        lst.remove(res[-1])
        switch = not switch
    return res

# Solution:
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#


✅ Generated Code (preview):

# Task:
Implement a function that sorts a list of integers using bubble sort.

# Related Examples:
return [] if len(array) == 0 el

Some weights of the model checkpoint at Salesforce/codegen-350M-multi were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (


✅ Generated Code:

# Task:
Write a function that counts the number of vowels in a string.

# Related Examples:
vowels = "aeiouAEIOU"
    n_vowels = sum(c in vowels for c in s)
    if s[-1] == 'y' or s[-1] == 'Y':
        n_vowels += 1
    return n_vowels

count = 0
    for i in range(0,len(s),2):
        if s[i] in "AEIOU":
            count += 1
    return count

return "".join([s for s in text if s.lower() not in ["a", "e", "i", "o", "u"]])

# Solution:
#
# Write a function that counts the number of vowels in a string.
#
# Write a function that returns the number of vowels in a string.
#
# Write a function that returns the number of vowels in a string.
#
# Write a function that returns the number of vowels in a string.
#
# Write a function that returns the number of vowels in a string.
#
# Write a function that returns the number of vowels in a string.
#
# Write a function that returns the number of vowels in a string.
#
# Write a function that returns the number of vowels in a stri

Some weights of the model checkpoint at Salesforce/codegen-350M-multi were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (


✅ Generated Code:

# Task:
Create a Python function that reverses a linked list.

# Related Examples:
odd = 0
    even = 0
    for i in lst1:
        if i%2 == 1:
            odd += 1
    for i in lst2:
        if i%2 == 0:
            even += 1
    if even >= odd:
        return "YES"
    return "NO"

if l == sorted(l) or l == sorted(l, reverse=True):
        return True
    return False

return [(e + 1) for e in l]

# Solution:
#
#     1
#     2
#     3
#     4
#     5
#     6
#     7
#     8
#     9
#   10
#   11
#   12
#   13
#   14
#   15
#   16
#   17
#   18
#   19
#   20
#   21
#   22
#   23
#   24
#   25
#   26
#   27
#   28
#   29
#   30
#   31
#   32
#   33
#   34
#   35
#   36
#   37
#   38
#   39
#   40
#   41
#   42
#   43
#   44
#   45
#   46
#   47
#   48
#   49
#   50
#   51
#   52
#   53
#   54
#   55
#   56
#   57
#   58
#   59
#   60
#   61
#   62
#   63
#   

✅ Generated Code (preview):

# Task:
Create a Python function that reverses a linked list.

# Related Examp